In [ ]:
import cv2
import os
import numpy as np
from sklearn.utils import shuffle
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential , Input, Model
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,Input,LSTM
from keras.models import Model
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Dropout, Flatten


from keras import optimizers
from tensorflow.python import keras

from sklearn.model_selection import train_test_split
from keras.layers.embeddings import Embedding

from keras.applications import ResNet50, VGG16

from keras.layers import Conv3D, MaxPooling3D, GlobalAveragePooling3D, Conv2D, BatchNormalization, MaxPooling2D, Activation, Convolution3D, ZeroPadding3D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD

EPOCHS = 10
BATCH_SIZE = 32
Train_Videos_dir = '/content/drive/My Drive/Training_set/Training'
Test_Videos_dir = '/content/drive/My Drive/Testing_set'

def Make_submission(predictions):
    results = []
    Myvideos = []

    for pred in predictions:
    #np.argmax return the index of the max element in the array
        if np.argmax(pred) == 0:
            results.append('2')
        elif np.argmax(pred) == 1:
            results.append('0')
        elif np.argmax(pred) == 2:
            results.append('1')
        elif np.argmax(pred) == 3:
            results.append('3')
        elif np.argmax(pred) == 4:
            results.append('4')

    Videos = np.load('testvideos.npy')
    for vid in Videos:
        Myvideos.append(vid)

    submission = pd.DataFrame({
        "Videos": Myvideos,
        "Label": results
    })
    submission.to_csv('Submit.csv', index=False)
    print('Done Saving Data You Can now Submit')

#encoding based on the training data
def Encoding_data(Folder_name):
    if Folder_name == 'Basketball':
        return np.array([1, 0, 0, 0, 0])
    if Folder_name == 'Diving':
        return np.array([0, 1, 0, 0, 0])
    if Folder_name == 'Jumping':
        return np.array([0, 0, 1, 0, 0])
    if Folder_name == 'Tennis':
        return np.array([0, 0, 0, 1, 0])
    if Folder_name == 'Walking':
        return np.array([0, 0, 0, 0, 1])

#takes directory of the data and return array of [frames, label]
def read_train_videos(Directory):
    All_videos = []
    All_Labels = []
    for fold in os.listdir(Directory):
        #Categorical enconding
        Label = Encoding_data(fold)
        Folder_path = os.path.join(Directory, fold)
        for vid in tqdm(os.listdir(Folder_path)):
            Myvideos = []
            Video_path = os.path.join(Folder_path, vid)
            cap = cv2.VideoCapture(Video_path)
            nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
            rem = nframe
            cnt = 0
            #iterates on 1 video
            while rem >= 16:
                ret, frame = cap.read()
                if frame is None:
                    break
                if cnt < 16:
                    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    final_frame = cv2.resize(frame, (112, 112))
                   
                    #final_frame=np.reshape(final_frame,(112,112,1))

                    Myvideos.append(final_frame)
                    cnt = cnt + 1
                else:
                    cnt = 0
                    All_videos.append(Myvideos)
                    All_Labels.append(Label)
                    Myvideos = []
                    rem = rem - 16

    FinalVideos, FinalLabels =  shuffle(All_videos, All_Labels)
    np.save("All_Videos.npy", FinalVideos)
    np.save('Labels.npy', FinalLabels)
    return FinalVideos, FinalLabels

def read_test_videos(Directory):
    All_videos = []
    array_indicies=[]
    total_sub_videos=0

    for vid in tqdm(os.listdir(Directory)):
        MyvideosList = []
        Video_path = os.path.join(Directory, vid)
        cap = cv2.VideoCapture(Video_path)
        nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        rem = nframe
        cnt = 0
        while rem>=16:
            ret, frame = cap.read()
            if frame is None:
                break
            if cnt<16:
                #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                final_frame = cv2.resize(frame, (112, 112))
                #final_frame=np.reshape(final_frame,(32,32,1))
                MyvideosList.append(final_frame)
                cnt=cnt+1
            else:
                cnt = 0
                All_videos.append(MyvideosList)
                MyvideosList = []
                total_sub_videos=total_sub_videos+1
                rem = rem - 16
        array_indicies.append(total_sub_videos) 
        total_sub_videos=0     
        
        

    np.save('array_indicies.npy',array_indicies) 
    np.save('Testvideos.npy',All_videos)
    return All_videos,array_indicies


def get_model():
    """ Return the Keras model of the network
    """
    BaseModel = VGG16(weights='imagenet', include_top=False)
    for layer in BaseModel.layers[:-1]:
      layer.trainable=False
    #################################
    model = Sequential()
    x = Input(shape=(16, 112, 112, 3))
    model=TimeDistributed((BaseModel))(x) 
    model=TimeDistributed((Conv2D(filters=64,kernel_size=(3,3),input_shape=(16, 112, 112, 3),padding='same')))(model)
    model=TimeDistributed((Conv2D(filters=64,kernel_size=(3,3),input_shape=(16, 112, 112, 3),padding='same')))(model)
    # model=LSTM(100)(model)
    model=TimeDistributed(Dropout(0.5))(model)
    model=Flatten()(model)
    # model=Dense(128, activation='relu')(model)
    model=Dense(32, activation='relu')(model)
    # model=Dense(128, activation='relu')(model)
    out=Dense(5, activation='softmax')(model)
    ################################
    model=Model(inputs = x, outputs = out)

    
    return model

def run_model(Model, TrainData, TrainY, TestData):
    opt = optimizers.adam(lr=0.00001)
    #optimizer='adam'
    Model.compile(loss="categorical_crossentropy", optimizer= opt , metrics=["accuracy"])
    print("aa")
    print(np.shape(TrainData))
    print(np.shape(TrainY))
    #TrainData=TrainData.reshape(TrainData.shape[0],224,224,25,1)
    print(np.shape(TrainData))

    #train_X,valid_X,train_label,valid_label=train_test_split(TrainData, TrainY, test_size=0.2, random_state=13)
    Model.fit(np.array(TrainData), np.array(TrainY), epochs=7, verbose=1, batch_size=8)
    #Model.fit(np.array(train_X),np.array(train_label), validation_data=(valid_X,valid_label),epochs=30, verbose=1, batch_size=32)
    print("Done training")
    predictions = Model.predict(np.array(TestData))
    print("Done testing")

    return predictions

if __name__ == '__main__':
    if os.path.exists('/content/drive/My Drive/All_Videoss.npy'):
        TrainData = np.load('/content/All_Videos.npy')
        Mylabels =  np.load('/content/Labels.npy')
        TestData = np.load('/content/Testvideos.npy',allow_pickle=True)
        indicies=np.load('/content/array_indicies.npy')
    else:
        TrainData, Mylabels = read_train_videos(Train_Videos_dir)
        TestData,indicies = read_test_videos(Test_Videos_dir)

    
    M = get_model()
    

 
 
    #softmax_L=Dense(5, activation='softmax', name='fc8')
    #Final_model=Model(N_model.input,softmax_L)
    #N_model=Dense(5, activation='softmax', name='fc8')(N_model)
    print(np.shape(TestData))
    if os.path.exists('predic.npy'):
        Predictions=np.load('predic.npy')
    else:
        Predictions = run_model(M, TrainData, Mylabels, TestData)
        np.save('predic.npy', Predictions)
    
    ind=0
  
    
    results=[]
    
    for i in range(0,len(indicies)):
        freq_arr=[0,0,0,0,0]
        for j in range(0,indicies[i]):
            pred=Predictions[ind+j]
            freq_arr[0]=freq_arr[0]+pred[1]
            freq_arr[1]=freq_arr[1]+pred[2]
            freq_arr[2]=freq_arr[2]+pred[0]
            freq_arr[3]=freq_arr[3]+pred[3]
            freq_arr[4]=freq_arr[4]+pred[4]
            #if np.argmax(pred) == 0:
             #   freq_arr[2]=freq_arr[2]+1
            #elif np.argmax(pred) == 1:
             #   freq_arr[0]=freq_arr[0]+1
            #elif np.argmax(pred) == 2:
             #   freq_arr[1]=freq_arr[1]+1
            #elif np.argmax(pred) == 3:
             #   freq_arr[3]=freq_arr[3]+1
            #elif np.argmax(pred) == 4:
             #   freq_arr[4]=freq_arr[4]+1

        results.append(np.argmax(freq_arr))
        

        ind=ind+indicies[i]


    #np.save('predic.npy', Predictions)
    #Make_submission(Predictions)
    videos_names=[]
    for vid in os.listdir(Test_Videos_dir):
        videos_names.append(vid)

    submission = pd.DataFrame({
        "Video": videos_names,
        "Label": results
    })
    submission.to_csv('Submit.csv', index=False)
    print('Done Saving Data You Can now Submit')
    
